#Analisador de Processo de Compra

Objetivo 1: Construir as mensagens dos emails para envio das notas de empenho

Objetivo 2: Listar os produtos fracassados e checar se todos os produtos homologados pela DG constam nas notas de empenho anexadas ao processo

Objetivo 3: Criar uma Planilha para acompanhamento das entregas dos produtos pelos fornecedores

Objetivo 4: Checar se todos os pagamentos aos fornecedores já foram feitos para poder arquivar o processo

In [1]:
!pip install nltk



In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [5]:
import PyPDF2

# Abrir o arquivo PDF
with open('/content/drive/MyDrive/Colab Notebooks/Processo_compras/SEI_03699_2024.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)

    # Extrair texto de todas as páginas
    textPDF = ''
    for page in reader.pages:
        textPDF += page.extract_text()

    # Obter o número de páginas
    num_pages = len(reader.pages)

# Exibir o número de páginas
print(f"O documento contém {num_pages} páginas.")

O documento contém 1451 páginas.


O processo é carregado para textPDF

In [6]:
#print(textPDF)

In [7]:
#capturar os dados principais das notas de empenho

import nltk
nltk.download('punkt')
# Dividir o texto em linhas
lines = nltk.tokenize.sent_tokenize(textPDF)


notas_de_empenho = [line for line in lines if "Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL" in line]

# Exibir as linhas correspondentes
for nota_emp in notas_de_empenho:
    print(nota_emp)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


1245Data e hora da consulta:
Usuário:  13/06/2024 16:46
        ***.929.604-**
Nota de Empenho
 
 
      Impressão Completa
Código Nome Moeda
Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL - (R$)
2024 NE 332
Natureza da Despesa Esfera UGR PTRES Plano Interno Fonte de RecursoCélula Orçamentária
1 167636 1000000000 339030 70226 ADM MATMOD
Data de Emissão Tipo Processo Taxa de Câmbio Valor
Código Nome
Descrição
Local da Entrega
Informação Complementar04/06/2024 Ordinário 3699/2024 0,0000 9.862,81
54.416.857/0001-16 L & C PARTICIPACOES E COMERCIO LTDA
AQUISIÇÃO DE MATERIAL FARMACOLÓGICO, HOSPITALAR E ODONTOLÓGICO PARA A SAMS - PREGÃO
ELETRÔNICO Nº 90023/2024 - PROCESSO SEI Nº 3699/2024.
1250Data e hora da consulta:
Usuário:  13/06/2024 16:50
        ***.929.604-**
Nota de Empenho
 
 
      Impressão Completa
Código Nome Moeda
Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL - (R$)
2024 NE 333
Natureza da Despesa Esfera UGR PTRES Plano Interno

In [8]:
len(notas_de_empenho)

5

In [9]:
import pprint
#text = notas_de_empenho[0]

# Criar o dicionário
lista_dict = []
data_dict = {}

for nota_emp in notas_de_empenho:
  # Extrair as informações
  lines = nota_emp.splitlines()

  # Criar o dicionário
  data_dict = {}

  # Extrair as informações
  lines = nota_emp.splitlines()

  # Encontrar e adicionar 'NE'
  for line in lines:
      if "NE" in line:
          parts = line.split()
          data_dict['NE'] = parts[parts.index('NE') + 1]

  # Encontrar e adicionar 'Empresa' e 'CNPJ'
  for line in lines:
      if "/0001-" in line:
          parts = line.split()
          data_dict['CNPJ'] = parts[0]
          data_dict['Empresa'] = ' '.join(parts[1:])
          break  # Para evitar múltiplas capturas

  # Encontrar e adicionar 'Valor' usando referências
  for line in lines:
      if "0,0000" in line:
          # Capturar o valor que vem depois de "0,0000 "
          valor = line.split("0,0000 ")[1].split("\n")[0].strip()
          data_dict['Valor'] = valor
          break  # Para evitar múltiplas capturas

  # Adicionar o dicionário à lista
  lista_dict.append(data_dict)
# Exibir o dicionário
pprint.pprint(lista_dict)

[{'CNPJ': '54.416.857/0001-16',
  'Empresa': 'L & C PARTICIPACOES E COMERCIO LTDA',
  'NE': '332',
  'Valor': '9.862,81'},
 {'CNPJ': '43.492.384/0001-10',
  'Empresa': 'ANA C REGIS',
  'NE': '333',
  'Valor': '311,14'},
 {'CNPJ': '22.077.847/0001-07',
  'Empresa': 'JOSE DANTAS DINIZ FILHO',
  'NE': '334',
  'Valor': '381,75'},
 {'CNPJ': '19.367.167/0001-13',
  'Empresa': 'COMERCIAL TARGET DE EPI E EPC LTDA',
  'NE': '335',
  'Valor': '1.592,50'},
 {'CNPJ': '30.843.402/0001-19',
  'Empresa': 'DOIS K COMERCIO E SERVICOS LTDA',
  'NE': '336',
  'Valor': '6.491,60'}]


In [10]:
#criação do DataFrame df_Notas_de_empenho com os dados do dicionario lista_dict

import pandas as pd
df_Notas_de_empenho = pd.DataFrame(lista_dict)
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81"
1,333,43.492.384/0001-10,ANA C REGIS,"311,14"
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75"
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50"
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60"


In [11]:
# prompt: capturar os emails que existirem no processo

import re

# Expressão regular para encontrar emails
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Encontrar todos os emails no texto
emails = re.findall(email_pattern, textPDF)
emails = list(set(emails))

# Remover os emails que forem do 'tre-rn'
filtered_emails = [item for item in emails if 'tre-rn' not in item]

email_minusculas = []
for string in filtered_emails:
    email_minusculas.append(string.lower())

email_minusculas = list(set(email_minusculas))
# Exibir os emails encontrados
pprint.pprint(email_minusculas)


['nacregis.parnamirim@gmail.com',
 'lcparticipacoescomercio@gmail.com',
 'dantasdiniz@gmail.com',
 'licitacoes@targethospitalar.com.br',
 'doiskcomercio@gmail.com',
 'anacregis.parnamirim@gmail.com',
 'acoescomercio@gmail.com']


In [12]:
#pegar os emails

import re

emails = []

for i in range (df_Notas_de_empenho.shape[0]):
  emails.append("nao_encontrado")

for i in range (df_Notas_de_empenho.shape[0]):
  # Texto extraído (exemplo)
  extracted_text = textPDF

  # String a ser encontrada
  company_name = df_Notas_de_empenho['Empresa'][i]
  stop_phrase = 'Solicito análise das propostas'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

  # Encontrar a posição da frase que limita a busca
  stop_index = extracted_text.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = extracted_text[:stop_index]

      # Encontrar a posição da empresa no texto
      company_match_name = re.search(re.escape(company_name), text_before_stop)
      #company_match_cnpj = re.search(re.escape(company_cnpj), text_before_stop)

      if company_match_name:
          # Obter o texto após a empresa
          text_after_company_name = text_before_stop[company_match_name.end():]

          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_name)

          if email_match:
              emails[i] = email_match.group()

  else:
      print("Frase de parada não encontrada no texto.")

for i in range (df_Notas_de_empenho.shape[0]):
  # Texto extraído (exemplo)
  extracted_text = textPDF

  # String a ser encontrada
  company_cnpj = df_Notas_de_empenho['CNPJ'][i]
  stop_phrase = 'Solicito análise das propostas'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

  # Encontrar a posição da frase que limita a busca
  stop_index = extracted_text.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = extracted_text[:stop_index]

      # Encontrar a posição da empresa no texto
      company_match_cnpj = re.search(re.escape(company_cnpj), text_before_stop)

      if company_match_cnpj:
          # Obter o texto após a empresa
          text_after_company_cnpj = text_before_stop[company_match_cnpj.end():]

          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_cnpj)

          if email_match:
            if emails[i] == "nao_encontrado":
              emails[i] = email_match.group()


  else:
      print("Frase de parada não encontrada no texto.")

df_Notas_de_empenho['Email'] = emails



In [13]:
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",nao_encontrado
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com


In [14]:
# gerar lista com empresas cujo email não foi encontrado

lista_empresas_sem_email = []
indices_sem_email = []

for i in range(df_Notas_de_empenho.shape[0]):
  if df_Notas_de_empenho['Email'][i] == "nao_encontrado":
    lista_empresas_sem_email.append(df_Notas_de_empenho['Empresa'][i])
    indices_sem_email.append(i)

#procurar os emails não encontrados
emails_encontrados = []
for empresa in lista_empresas_sem_email:
  empresa = empresa.split()
  for palavra in empresa:
        for string in email_minusculas:
            if palavra.lower() in string.lower():
                emails_encontrados.append(string)

emails_encontrados = list(set(emails_encontrados))
print(emails_encontrados)

for i in range(len(indices_sem_email)):
    df_Notas_de_empenho['Email'][indices_sem_email[i]] = emails_encontrados[i]


df_Notas_de_empenho

['dantasdiniz@gmail.com']


,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com


In [15]:

# Função para encontrar a linha do email no texto extraído
def find_email_line(email):
    # Regex para encontrar o email no texto extraído
    match = re.search(re.escape(email), extracted_text)
    if match:
        # Obter a linha onde o email foi encontrado
        lines = extracted_text.splitlines()
        for line in lines:
            if email in line:
                return line  # Retorna a linha correspondente
    return None  # Retorna None se o email não for encontrado

# Criar nova coluna 'Verif_email' com as linhas correspondentes
df_Notas_de_empenho['Verif_email'] = df_Notas_de_empenho['Email'].apply(find_email_line)

# Exibir o DataFrame atualizado

df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Email,Verif_email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM,"LCPARTICIPACOESCOMERCIO@GMAIL.COM , TELEFONE: ..."
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com,E-mail: a nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com,Diniz Dantas <dantasdiniz@gmail.com> 14 de jun...
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br,(11) 5461-1551 (11) 95146-8131 e-mail: licitac...
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com,E-mail: doiskcomercio@gmail.com


In [16]:
from IPython.display import display, HTML

# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Antes de continuar, checar o email na coluna do Email do DataFrame com o que aparece na coluna Verif_email no mesmo DataFrame!</h1>'
text2 = '<h1 style="color: red;">Caso seja necessário insira manualmente no DataFrame os emails que o programa não conseguiu detectar ou que esteja escrito errado.</h1>'
text3 = '<h1 style="color: red;">Foi colocado na célula abaixo um print errado para o código parar de executar.</h1>'
# Exibir o texto
display(HTML(text3))
display(HTML(text1))
display(HTML(text2))

In [17]:
print(parar)

NameError: name 'parar' is not defined

In [18]:
df_Notas_de_empenho['Email'][1] = "anacregis.parnamirim@gmail.com" #email sendo corrigido manualmente


In [19]:
from datetime import datetime

# Obter a data e hora atuais
now = datetime.now()

# Extrair dia, mês e ano
day = now.day
month = now.month
year = now.year

df_Notas_de_empenho['Mensagem'] = "escrever"

for i in range(df_Notas_de_empenho.shape[0]):
  df_Notas_de_empenho['Mensagem'][i] = f"""email: {df_Notas_de_empenho['Email'][i]}

  Prezados(as) Senhores(as),
      Encaminhamos a Nota de Empenho número {year}NE{df_Notas_de_empenho['NE'][i]} (em anexo), à empresa {df_Notas_de_empenho['Empresa'][i]}.
      Informamos que o prazo para entrega do material é de até 30 (trinta) dias corridos, contados a partir da data de envio
      desta Nota de Empenho.
      Solicitamos especial atenção quanto ao PRAZO, LOCAL e HORÁRIO para entrega do material.
      A entrega deverá ser realizada na Seção de Assistência Médica e Saúde Ocupacional (SAMS), localizada na Sede
      Administrativa do TRE/RN. Endereço: Av. Rui Barbosa, 215. Tirol, Natal-RN. CEP: 59.015-290. Telefone: (84) 3654-
      5399, nos horários de segunda a quinta-feira, das 13 às 18 horas ou na sexta-feira, das 8 às 13 horas.
      FAVOR CONFIRMAR RECEBIMENTO DESTE E-MAIL enviando resposta para o email sams@tre-rn.jus.br
      Atenciosamente,
      Flávio Roberto Guerra Seabra
      SAMS - TRE-RN, {day}/{month}/{year}.
    """
#df_Notas_de_empenho

In [20]:
#GERA ARQUIVO TXT COM OS TEXTOS DOS EMAILS PRONTOS PARA SEREM ENVIADOS

# Nome do arquivo de saída
output_file = 'textos_emails.txt'

# Abrir o arquivo para escrita
with open(output_file, 'w', encoding='utf-8') as file:
    # Loop através das linhas do DataFrame
    i = 0
    for index, row in df_Notas_de_empenho.iterrows():
        # Criar um parágrafo para cada linha
        paragraph = df_Notas_de_empenho['Mensagem'][i]
        i += 1
        # Escrever o parágrafo no arquivo
        file.write(paragraph)
        # Adicionar uma linha em branco entre os parágrafos
        file.write("\n")

print(f"Dados exportados para '{output_file}' com sucesso!")

Dados exportados para 'textos_emails.txt' com sucesso!


In [21]:
from IPython.display import display, HTML

# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Neste ponto o Objetivo 1 deve ter sido obtido</h1>'
text2 = '<h1 style="color: red;">Verifique se o arquivo "textos_emails.txt" foi criado .</h1>'

# Exibir o texto
display(HTML(text1))
display(HTML(text2))

Início de captura dos números dos itens Desertos e Fracassados

In [22]:
import re

# Texto extraído (exemplo)
extracted_text = textPDF

# Definindo o padrão para capturar o trecho desejado
pattern = r'(Pregão eletrônico.*?os demais itens lograram êxito e aguardam adjudicação)'

# Encontrar o trecho entre os padrões
match = re.search(pattern, extracted_text, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    parecer_ajdg_homolog = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [23]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Supondo que 'text' é o conteúdo extraído do PDF
tokens = word_tokenize(parecer_ajdg_homolog)

# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
parecer_ajdg_homolog_tokenizado = [word for word in tokens if word.lower() not in stop_words]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
parecer_ajdg_homolog_tokenizado = [s for s in parecer_ajdg_homolog_tokenizado if s != ',']


In [25]:
# Lista para armazenar os números dos itens desertos
lista_desertos = []

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if parecer_ajdg_homolog_tokenizado[i] == 'item':
      parecer_ajdg_homolog_tokenizado = 'itens'

capturar = False

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if parecer_ajdg_homolog_tokenizado[i] == 'itens' and parecer_ajdg_homolog_tokenizado[i+1].isdigit() and len(lista_desertos) == 0:
      capturar = True

    if parecer_ajdg_homolog_tokenizado[i].isdigit() and capturar == True:
      lista_desertos.append(parecer_ajdg_homolog_tokenizado[i])
    else:
      if len(lista_desertos) > 0:
        capturar = False

print(lista_desertos)

['22', '35', '36', '38', '40', '41', '42', '44', '45', '47', '49', '50', '52', '53', '60', '61', '63', '66', '76', '77', '83', '85', '87']


In [26]:
# Lista para armazenar os números dos itens desertos
lista_fracassados = []

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if parecer_ajdg_homolog_tokenizado[i] == 'item':
      parecer_ajdg_homolog_tokenizado = 'itens'

capturar = False

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if (parecer_ajdg_homolog_tokenizado[i] == 'desertos' or parecer_ajdg_homolog_tokenizado[i] == 'deserto'):
      capturar = True

    if parecer_ajdg_homolog_tokenizado[i].isdigit() and capturar == True:
      lista_fracassados.append(parecer_ajdg_homolog_tokenizado[i])


print(lista_fracassados)

['20', '74', '84']


Final de captura dos números dos itens Desertos e Fracassados

listas: **lista_fracassados** e **lista_desertos**

Iniciar capturar todos os numeros dos itens do termo de referencia

In [27]:
import re

# Texto extraído (exemplo)
extracted_text = textPDF

# Definindo o padrão para capturar o trecho desejado
pattern = r'(Definição do objeto.*?O objeto desta contratação não se enquadra como sendo de bem de luxo)'

# Encontrar o trecho entre os padrões
match = re.search(pattern, extracted_text, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    tabela_produtos = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [28]:
lista_textos_para_remover = [ "Câmara Nacional de Modelos de Licitações e Contratos da Consultoria-Geral da União",
                              "Aprovado pela Secretaria de Gestão e Inovação",
                              "UASG 70008"
                            ]

for texto in lista_textos_para_remover:
  tabela_produtos = tabela_produtos.replace(texto, "")

for i in range(1, num_pages + 1):
  tabela_produtos = tabela_produtos.replace(f"/ pg. {i}", "")

In [29]:
print(tabela_produtos)

Definição do objeto
1.1.      Aquisição de material hospitalar e odontológico, nos termos da tabela abaixo, conforme
condições e exigências estabelecidas neste instrumento.
ITEM Especificações CATMAT Unidade de Medida Quant.
1Absorvente higiênico, tamanho normal, 
sem abas, cobertura 100%
algodão. 230246Embalagem com 
8 unidades 10
2Álcool hidratado 70%
líquido, uso Farmacológico. 269941 Litro 10
Bolsa térmica gel, que
molda-se perfeitamente ao
corpo e não congela,
mesmo quando man tida
constantemente no
freezer, reu tilizável, com
gel térmico não tóxico,
para compressas quente e
gelada, podendo ser
Termo de referência (0032434)         SEI 03699/2024  Termo de Referência 5/2024

Modelo de Aquisições — Atualização: maio/2023
2 de 313resfriada no freezer,
aquecida no micro-ondas
ou na água quente.
Tamanho grande
(Comprimento: 54 cm.
Altura: 25 cm). 433757 Unidade 5
4Bolsa térmica gel, que
molda-se perfeitamente ao
corpo e não congela,
mesmo quando man tida
constantemente no
freezer, reu

In [30]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Supondo que 'text' é o conteúdo extraído do PDF
tokens = word_tokenize(tabela_produtos)

# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
tabela_produtos_tokenizada = [word for word in tokens if word.lower() not in stop_words]
tabela_produtos_tokenizada = [item for item in tabela_produtos_tokenizada if 'UASG' not in item]

tabela_produtos_tokenizada

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Definição',
 'objeto',
 '1.1',
 '.',
 'Aquisição',
 'material',
 'hospitalar',
 'odontológico',
 ',',
 'termos',
 'tabela',
 'abaixo',
 ',',
 'conforme',
 'condições',
 'exigências',
 'estabelecidas',
 'neste',
 'instrumento',
 '.',
 'ITEM',
 'Especificações',
 'CATMAT',
 'Unidade',
 'Medida',
 'Quant',
 '.',
 '1Absorvente',
 'higiênico',
 ',',
 'tamanho',
 'normal',
 ',',
 'abas',
 ',',
 'cobertura',
 '100',
 '%',
 'algodão',
 '.',
 '230246Embalagem',
 '8',
 'unidades',
 '10',
 '2Álcool',
 'hidratado',
 '70',
 '%',
 'líquido',
 ',',
 'uso',
 'Farmacológico',
 '.',
 '269941',
 'Litro',
 '10',
 'Bolsa',
 'térmica',
 'gel',
 ',',
 'molda-se',
 'perfeitamente',
 'corpo',
 'congela',
 ',',
 'man',
 'tida',
 'constantemente',
 'freezer',
 ',',
 'reu',
 'tilizável',
 ',',
 'gel',
 'térmico',
 'tóxico',
 ',',
 'compressas',
 'quente',
 'gelada',
 ',',
 'podendo',
 'Termo',
 'referência',
 '(',
 '0032434',
 ')',
 'SEI',
 '03699/2024',
 'Termo',
 'Referência',
 '5/2024',
 'Modelo',
 'Aquisiçõ

In [31]:
#Obter o número do último produto da lista

filtered_list = [s for s in tabela_produtos_tokenizada if re.match(r'^[1-9][0-9A-Z][a-zA-Z]', s)]
filtered_list = [s for s in filtered_list if not (len(s) <= 5 or 'UASG' in s)]
maior = max(filtered_list)
ultimo_produto = ''.join(filter(str.isdigit, maior))
ultimo_produto = int(ultimo_produto)
print(ultimo_produto)


87


In [32]:
# Criar a nova lista com valores de 1 a ultimo_produto, excluindo os números da lista de exclusão
lista_desertos = [int(s) for s in lista_desertos]
lista_fracassados = [int(s) for s in lista_fracassados]

# Criar lista com os produtos que não foram homologados
lista_remover = lista_desertos + lista_fracassados

# Clista com os números dos produtos homologados
produtos_homologados = [i for i in range(1, int(ultimo_produto) + 1) if i not in lista_remover] # Changed ultimo_produto to int(ultimo_produto)
print(produtos_homologados)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 39, 43, 46, 48, 51, 54, 55, 56, 57, 58, 59, 62, 64, 65, 67, 68, 69, 70, 71, 72, 73, 75, 78, 79, 80, 81, 82, 86]


Obtida a lista de produtos homologados: **produtos_homologados**

Iniciar a checagem se os produtos homologados estão nas notas de empenho

In [33]:
#capturar as notas de empenho para a lista notas_de_empenho_completas

import re

# Texto extraído (exemplo)
extracted_text = textPDF
notas_de_empenho_completas = []

for i in range (df_Notas_de_empenho.shape[0]):
  num_nota_de_empenho = df_Notas_de_empenho['NE'][i]
  # Definindo o padrão para capturar o trecho desejado
  pattern = rf'(2024 NE {num_nota_de_empenho}.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO)'

  # Encontrar o trecho entre os padrões
  match = re.search(pattern, extracted_text, re.DOTALL)

  if match:
      # Obter o texto completo entre os padrões
      nota_empenho = match.group(1).strip()  # O texto entre os dois padrões
      notas_de_empenho_completas.append(nota_empenho)
  else:
      print("Trecho não encontrado.")



In [34]:
#inserir as notas de empenho tokenizadas e sem stopwords no DataFrame

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df_Notas_de_empenho['NE_Completa'] = ""

for i in range(len(notas_de_empenho_completas)):
  nltk.download('punkt')
  nltk.download('stopwords')

  # Supondo que 'text' é o conteúdo extraído do PDF
  tokens = word_tokenize(notas_de_empenho_completas[i])

  # Remover stopwords (palavras comuns que não têm significado relevante)
  stop_words = set(stopwords.words('portuguese'))
  df_Notas_de_empenho['NE_Completa'][i] = [word for word in tokens if word.lower() not in stop_words]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Packag

In [35]:
#Capturar os números dos itens de cada nota de empenho e inserir no DataFrame

import numpy as np

df_Notas_de_empenho['itens_da_NE'] = ''

for i in range(df_Notas_de_empenho.shape[0]):
  for j in range(len(df_Notas_de_empenho['NE_Completa'][i])):
    if (df_Notas_de_empenho['NE_Completa'][i][j] == 'Item') and (df_Notas_de_empenho['NE_Completa'][i][j+1] == 'compra'):
      df_Notas_de_empenho['itens_da_NE'][i] += df_Notas_de_empenho['NE_Completa'][i][j+3]
      df_Notas_de_empenho['itens_da_NE'][i] += ', '

# Remove the last comma and space
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.rstrip(', ')

# Split the string into a list of strings
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.split(', ')

# Convert the list of strings to a list of integers
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].apply(lambda x: [int(i) for i in x])

In [36]:
#dropar colunas que não são mais necessárias

colunas_dropar = ['Verif_email', 'Mensagem']
df_Notas_de_empenho = df_Notas_de_empenho.drop(colunas_dropar, axis=1)


In [37]:
#Criar uma lista com os produtos que foram homologados mas não estão em nenhuma Nota de Empenho

produtos_homologados_nao_presentes_nas_NE = produtos_homologados.copy()

for lista in df_Notas_de_empenho['itens_da_NE']:
  for num in lista:
    if num in produtos_homologados_nao_presentes_nas_NE:
      produtos_homologados_nao_presentes_nas_NE.remove(num)

#print(f'Produtos que foram homologados mas não estão em nenhuma NE: \n{produtos_homologados_nao_presentes_nas_NE}')

In [38]:
from IPython.display import display, HTML
if (len(produtos_homologados_nao_presentes_nas_NE) >0):
  # Texto em vermelho e com fonte maior
  text1 = '<h1 style="color: red;">Aparentemente os seguintes produtos que foram homologados não estão em nenhuma Nota de Empenho</h1>'
  text2 = '<h1 style="color: red;">FAVOR CHECAR!!!</h1>'
  # Exibir o texto
  display(HTML(text1))
  display(HTML(text2))
  print()
  print(f'Produtos que foram homologados mas não estão em nenhuma NE: \n{produtos_homologados_nao_presentes_nas_NE}')


Produtos que foram homologados mas não estão em nenhuma NE: 
[1, 9, 10, 30, 31, 54, 55, 56, 57, 78, 79]


In [39]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 2 atingido</h1>'

# Exibir o texto
display(HTML(text1))

#Início de criação da planilha de acompanhamento das entregas (Objetivo 3)

In [40]:
#Cria df_para_planilha

resposta = input("Deseja criar a Planilha de Controle de Entregas? (s/n): ")

if resposta.lower() == 's':
  colunas_dropar = ['NE_Completa', 'itens_da_NE']
  df_para_planilha = df_Notas_de_empenho.drop(colunas_dropar, axis=1)
  df_para_planilha['Data da Entrega'] = ''
  df_para_planilha['No. da Nota Fiscal'] = ''
  df_para_planilha['Pendências na Entrega'] = ''
  df_para_planilha['Enviado para pagamento em:'] = ''
  df_para_planilha['Pago em:'] = ''
  df_para_planilha.to_excel('Planilha de Controle de Entregas.xlsx', index=False)

Deseja criar a Planilha de Controle de Entregas? (s/n): s


In [41]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 3 atingido</h1>'

# Exibir o texto
display(HTML(text1))

#Início da checagem de pagamentos (Objetivo 4)

In [42]:
# prompt: separe textPDF e uma outra variavel textPDF2 onde textPDF2 terá apenas a parte depois de onde aparece pela primeira vez 'Envio de Nota de Empenho'

textPDF2 = textPDF.split('Envio de Nota de Empenho', 1)
if len(textPDF2) > 1:
  textPDF2 = textPDF2[1]
else:
  textPDF2 = ""


In [43]:
#capturar comprovantes de pagamento

#capturar as notas de empenho para a lista notas_de_empenho_completas

import re

# Texto extraído (exemplo)
lista_comprovantes_pagamento = []

for i in range (df_Notas_de_empenho.shape[0]):
  parte_cnpj = df_Notas_de_empenho['CNPJ'][i]
  parte_cnpj = parte_cnpj[8:]
  empresa = df_Notas_de_empenho['Empresa'][i]
  # Definindo o padrão para capturar o trecho desejado
  pattern = rf'({parte_cnpj} - {empresa}.*?QUITADA CONFORME INSTRUCAO NORMATIVA)'

  # Encontrar o trecho entre os padrões
  match = re.search(pattern, textPDF2, re.DOTALL)

  if match:
      # Obter o texto completo entre os padrões
      comprovante = match.group(1).strip()  # O texto entre os dois padrões
      lista_comprovantes_pagamento.append(comprovante)
  else:
      print("Trecho não encontrado.")


In [44]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

for i in range(len(lista_comprovantes_pagamento)):
  # Supondo que 'text' é o conteúdo extraído do PDF
  tokens = word_tokenize(lista_comprovantes_pagamento[i])

  # Remover stopwords (palavras comuns que não têm significado relevante)
  stop_words = set(stopwords.words('portuguese'))
  comprovante_pagamento_tokenizado = [word for word in tokens if word.lower() not in stop_words]
  lista_comprovantes_pagamento[i] = comprovante_pagamento_tokenizado

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
lista_comprovantes_pagamento[2]

['47/0001-07',
 '-',
 'JOSE',
 'DANTAS',
 'DINIZ',
 'FILHO',
 'TITULO',
 'CREDITO',
 ':',
 '2024NP000504',
 'DATA',
 'VENCIMENTO',
 ':',
 '05Jul24',
 'OBSERVACAO',
 'DOCUMENTO',
 'EMITIDO',
 'SIAFI-WEB',
 ',',
 'FRUTO',
 'EMISSÃO',
 'ORDEM',
 'PAGAMENTO',
 '.',
 'CONTINUA',
 '...',
 'LANCADO',
 ':',
 '04667539429',
 '-',
 'SOLANGE',
 'COSTA',
 'UG',
 ':',
 '070008',
 '21Jun24',
 '09:51',
 'PF1-AJUDA',
 'PF3=SAI',
 'PF4=ESPELHO',
 'PF12=RETORNA',
 'Pagamento',
 '(',
 '0052483',
 ')',
 'SEI',
 '03699/2024',
 '/',
 'pg',
 '.',
 '1309',
 '__',
 'SIAFI2024-DOCUMENTO-CONSULTA-CONDOC',
 '(',
 'CONSULTA',
 'DOCUMENTO',
 ')',
 '___________________',
 '21/06/24',
 '09:52',
 'NS',
 'USUARIO',
 ':',
 'SOLANGE',
 'COSTA',
 'DATA',
 'EMISSAO',
 ':',
 '21Jun24',
 'VALORIZACAO',
 ':',
 '21Jun24',
 'NUMERO',
 ':',
 '2024NS003033',
 'UG/GESTAO',
 'EMITENTE',
 ':',
 '070008',
 '/',
 '00001',
 '-',
 'TRIBUNAL',
 'REGIONAL',
 'ELEITORAL',
 'RIO',
 'G.DO',
 'N',
 'FAVORECIDO',
 ':',
 '22077847/0001-07',
 '-

In [46]:
import re

datas_pagamento = []

def encontrar_sequencia_apos_lancada(lista_palavras):

    padrao = re.compile(r'\d{2}[A-Z][a-z]{2}\d{2}')

    AGENCIA_encontrada = False

    # Percorrer a lista de palavras
    for palavra in lista_palavras:
        if palavra == 'AGENCIA':
            AGENCIA_encontrada = True
            continue

        if AGENCIA_encontrada and padrao.match(palavra):
            return palavra

    # Retornar None se não encontrar
    return None

# Exemplo de uso
for i in range(len(lista_comprovantes_pagamento)):
  resultado = encontrar_sequencia_apos_lancada(lista_comprovantes_pagamento[i])
  print(f'{lista_comprovantes_pagamento[i][0]}: {resultado}')
  datas_pagamento.append(f'{lista_comprovantes_pagamento[i][0]}: {resultado}')


57/0001-16: 28Jun24
84/0001-10: 21Jun24
47/0001-07: 21Jun24
67/0001-13: 08Ago24
02/0001-19: 12Jul24


In [47]:
# def limpar_cnpj(cnpj):
#     # Usar expressão regular para remover pontos, traços e barras
#     cnpj_limpo = re.sub(r'[./-]', '', cnpj)
#     return cnpj_limpo

# pagos = []

# for comprovante in lista_comprovantes_pagamento:
#   for i in range(df_Notas_de_empenho.shape[0]):
#     cnpj = df_Notas_de_empenho['CNPJ'][i]
#     cnpj = limpar_cnpj(cnpj)
#     for item in comprovante:
#       item = limpar_cnpj(cnpj)
#       if cnpj == item:
#         pagos.append(cnpj)

# pagos = list(set(pagos))
# pagos

In [48]:
# def resturar_cnpj(string):
#     # Inserir o ponto entre a posição 2 e 3
#     cnpj = string[:2] + '.' + string[2:]
#     # Inserir o ponto entre a posição 7 e 8
#     cnpj = cnpj[:6] + '.' + cnpj[6:]
#     # Inserir a barra entre a posição 11 e 12
#     cnpj = cnpj[:10] + '/' + cnpj[10:]
#     cnpj = cnpj[:15] + '-' + cnpj[15:]
#     return cnpj

# pagos_restaurados = []

# for cnpj in pagos:
#   pagos_restaurados.append(resturar_cnpj(cnpj))



In [49]:

# def encontrar_sequencia(lista):
#     # Expressão regular para a sequência de 7 caracteres
#     padrao = re.compile(r'^\d{2}[A-Z][a-z]{2}\d{2}$')

#     # Percorrer a lista de trás pra frente
#     for item in reversed(lista):
#         # Verificar se o item corresponde ao padrão
#         if padrao.match(item):
#             return item

#     # Retornar None se não encontrar
#     return None

# datas_pagamento = []

# for i in range(len(lista_comprovantes_pagamento)):
#   #for j in range(len(lista_comprovantes_pagamento[i])):
#       data = encontrar_sequencia(lista_comprovantes_pagamento[i])
#       if data is not None:
#         datas_pagamento.append(f'{pagos_restaurados[i]}: {data}')

# df_Notas_de_empenho['Data_Pag'] = ''
# for i in range(df_Notas_de_empenho.shape[0]):
#   for j in range(len(datas_pagamento)):
#     if (df_Notas_de_empenho['CNPJ'][i] == datas_pagamento[j].split(':')[0]) and len(datas_pagamento[j].split(':')[1])> 3:
#       df_Notas_de_empenho['Data_Pag'][i] = datas_pagamento[j].split(':')[1]


# df_Notas_de_empenho

In [50]:
datas_pagamento

['57/0001-16: 28Jun24',
 '84/0001-10: 21Jun24',
 '47/0001-07: 21Jun24',
 '67/0001-13: 08Ago24',
 '02/0001-19: 12Jul24']

In [51]:
dropar = ['NE_Completa', 'itens_da_NE', 'Email']
df_Notas_de_empenho = df_Notas_de_empenho.drop(dropar, axis=1)

df_Notas_de_empenho['Data_Pag'] = 'Não'

for i in range(df_Notas_de_empenho.shape[0]):
  for j in range(len(datas_pagamento)):
    if datas_pagamento[j].split(':')[0] in df_Notas_de_empenho['CNPJ'][i]:
      df_Notas_de_empenho['Data_Pag'][i] = datas_pagamento[j].split(':')[1]



In [52]:
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Data_Pag
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",28Jun24
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",21Jun24
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",21Jun24
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",08Ago24
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",12Jul24


In [53]:
df_Notas_de_empenho.to_excel('Planilha de Controle de Pagamentos.xlsx', index=False)

In [54]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 4 atingido</h1>'
text1 = '<h1 style="color: red;">Verifique se o arquivo "Planilha de Controle de Pagamentos.xlsx" foi criado.</h1>'

# Exibir o texto
display(HTML(text1))